In [1]:
import numpy as np 
import pandas as pd 

In [2]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [3]:
df = pd.read_csv('gurgaon_properties_post_feature_selection.csv')

In [4]:
df

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category,price
0,0.0,67.0,2,2,1.0,3.0,814.000000,0,0,1,1.0,2.0,0.45
1,0.0,29.0,2,2,1.0,2.0,588.000000,0,0,0,1.0,1.0,0.50
2,0.0,96.0,2,2,3.0,1.0,538.000000,0,0,1,1.0,0.0,0.40
3,0.0,59.0,2,2,2.0,1.0,1086.000000,0,0,1,2.0,1.0,1.47
4,0.0,92.0,2,2,3.0,4.0,1217.000000,0,0,1,1.0,2.0,0.70
5,0.0,33.0,2,2,3.0,1.0,654.000000,0,0,1,1.0,2.0,0.41
6,0.0,89.0,3,3,3.0,1.0,1357.000000,0,0,1,1.0,2.0,2.00
7,0.0,4.0,3,4,3.0,1.0,2072.000000,0,0,1,2.0,0.0,1.80
8,0.0,86.0,2,4,4.0,4.0,1350.000000,0,0,1,2.0,1.0,1.10
9,0.0,76.0,3,3,2.0,0.0,2813.000000,1,0,1,2.0,0.0,4.75


In [5]:
x=df.drop(columns=['price'])
y = df['price']

In [6]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder , StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

In [7]:
columns_to_encode = ['sector','balcony','agePossession','furnishing_type','luxury_category','floor_category']

In [8]:
y_transformed = np.log1p(y)

In [9]:
preprocessor = ColumnTransformer

In [10]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['property_type', 'bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OneHotEncoder(drop='first'), columns_to_encode)
    ], 
    remainder='passthrough'
)

In [11]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
#     ('regressor', LinearRegression())
    ('regressor', SVR(kernel ='rbf'))
])

In [12]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, x, y_transformed, cv=kfold, scoring='r2')

In [13]:
scores.mean()

0.887001277407464

In [14]:
scores.std()

0.013861171619216846

In [15]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y_transformed,test_size=0.2,random_state=42)
pipeline.fit(x_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['property_type', 'bedRoom',
                                                   'bathroom', 'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['sector', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category'])])),
                ('regressor', SVR())])

In [16]:
y_pred = pipeline.predict(x_test)
y_pred = np.expm1(y_pred)
from sklearn.metrics import mean_absolute_error
mean_absolute_error(np.expm1(y_test),y_pred)

0.5785910214289308

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3554 entries, 0 to 3553
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   property_type    3554 non-null   float64
 1   sector           3554 non-null   float64
 2   bedRoom          3554 non-null   int64  
 3   bathroom         3554 non-null   int64  
 4   balcony          3554 non-null   float64
 5   agePossession    3554 non-null   float64
 6   built_up_area    3554 non-null   float64
 7   servant room     3554 non-null   int64  
 8   store room       3554 non-null   int64  
 9   furnishing_type  3554 non-null   int64  
 10  luxury_category  3554 non-null   float64
 11  floor_category   3554 non-null   float64
 12  price            3554 non-null   float64
dtypes: float64(8), int64(5)
memory usage: 361.1 KB
